### LangChain Essentials

# Prompts templates for OpenAI & Llama - LangChain #2

One of the most fundamental and underrated pieces of langchain is prompting, prompts are what we would send to our LLM to allow them to answer in a formulated response, super useful if we want to do something such as making NPCs speak with a specific accent or creating a song with a specifc twist in mind.

In [1]:
"""            
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
"""

'            \nAnswer the question based on the context below,                 }\nif you cannot answer the question using the                     }--->  (Rules) For Our Prompt\nprovided information answer with "I don\'t know"                 }\n\nContext: Aurelio AI is an AI development studio                 }\nfocused on the fields of Natural Language Processing (NLP)      }\nand information retrieval using modern tooling                  }--->   Context AI has\nsuch as Large Language Models (LLMs),                           }\nvector databases, and LangChain.                                }\n\nQuestion: Does Aurelio AI do anything related to LangChain?     }--->   User Question\n\nAnswer:                                                         }--->   AI Answer\n'

## Choosing your Model

This example is split into two versions - The [Ollama version], allowing us to run our LLM locally without needing any external services or API keys. The [OpenAI version] uses the OpenAI API and requires an OpenAI API key.

## Initializing Llama 3.2

We start by initializing the 1B parameter Llama 3.2 model, fine-tuned for instruction following. We pull the model from Ollama by switching to our terminal and executing:

ollama pull llama3.2:1b-instruct-fp16

Once the model has finished downloading, we initialize it in LangChain using the ChatOllama class:

In [2]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

# initialize another LLM with temperature 0.9, this makes the LLM more creative
creative_llm = ChatOllama(temperature=0.9, model=model_name)

## Basic Prompting

Here we can see how the AI will appoach our question, as you can see we have a formulated response, if the context has the answer, then use the context to answer the question, if not, say I don't know, then we also have context and question which are being passed into this similarly to paramaters in a function.

In [3]:
Prompt = """
Answer the question based on the context below,                 
if you cannot answer the question using the                   
provided information answer with "I don't know"

context: {context}

question: {query}
"""
Context = """
Context: Aurelio AI is an AI development studio                 
focused on the fields of Natural Language Processing (NLP)      
and information retrieval using modern tooling                  
such as Large Language Models (LLMs),                           
vector databases, and LangChain. Owned by James Briggs
"""

Firstly we want to create our template we will use for our questions.

In [4]:
from langchain.prompts import ChatPromptTemplate

# Passing the template to the LangChain model.
prompt_template = ChatPromptTemplate.from_template(Prompt)

Apply any question into here!

In [5]:
Question = "Who owns Aurelio AI"

This is where we're converting the message into the correct format for our function.

In [6]:
messages = prompt_template.format_messages(prompt=Prompt, query=Question, context=Context)

Now we will retrieve our answer and showcase the answer:

In [7]:
answer = llm(messages)
print(answer.content)

C:\Users\Joshu\AppData\Local\Temp\ipykernel_53560\1989704094.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = llm(messages)


I don't know.


That works for the basic chatbots, however what if we want to give some questions and answers so that the AI can have a rough idea of what we want it to say?

## Old Few Shot Prompting

This section will cover Few Shot Prompt Templates allowing us to be able to go more in depth with our AI

In [8]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

1. Firstly we want to add a example template, this will tell the AI how we would like to format our inputs and outputs, in this case, the questions and answers.
2. Secondly we want to add a prompt template, this again will be used to tell the AI how we would like it to use the examples and answer the questions.

In [9]:
# Define the example template
example_template = """Input: {input}
Output: {output}"""

# Define the prompt template
prompt_template = "Given the following examples, complete the task:"

Here is a list of questions and answers, here you can see that we're trying to match words based on their similarity to other words.

In [10]:
Examples = [
        {"input": "Clouds, Sky, Sun, Space, Planets", "output": "Galaxy"},
        {"input": "Eyes, Face, Human, Animal", "output": "Species"},
    ]

Now we want to make a few shot template where we can feed all this data into.

In [11]:
few_shot_prompt = FewShotPromptTemplate(
    examples=Examples,
    example_prompt=PromptTemplate.from_template(example_template),
    prefix = prompt_template,
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    example_separator="\n\n",
)

Here we can carry on with our question to see what it thinks is similar too.

In [12]:
# Format the prompt with input
formatted_prompt = few_shot_prompt.format(input="Fitness, Health, Lifestyle")
print(formatted_prompt)

Given the following examples, complete the task:

Input: Clouds, Sky, Sun, Space, Planets
Output: Galaxy

Input: Eyes, Face, Human, Animal
Output: Species

Input: Fitness, Health, Lifestyle
Output:


Now we need to run this through our model and see what the output is!

In [13]:
from langchain.schema import HumanMessage

In [14]:
output = creative_llm(messages=[HumanMessage(content=formatted_prompt)])
print(output.content)

I'd be happy to help you complete the tasks.

Here are the completed examples:

 Input: Clouds, Sky, Sun, Space, Planets
 Output: Galaxy

 Input: Eyes, Face, Human, Animal
 Output: Kingdom

 Input: Fitness, Health, Lifestyle
 Output: Species


However that is considered the old way of doing few shot prompts, here below we can see a newer way which is way easier.

## New Few Shot Prompting

In [15]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [16]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
# This is our few shot template used to feed our examples into the LLM
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=Examples,
)
# Here we can view the format of our content down below
print(few_shot_prompt.format())

Human: Clouds, Sky, Sun, Space, Planets
AI: Galaxy
Human: Eyes, Face, Human, Animal
AI: Species


In [17]:
# Here we are formatting our final prompt for the AI to use, take note that the input needs to be the same keyword as the input below.
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a poem writer."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a poem writer.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Clouds, Sky, Sun, Space, Planets', 'output': 'Galaxy'}, {'input': 'Eyes, Face, Human, Animal', 'output': 'Species'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [18]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=final_prompt, llm=creative_llm)

chain.invoke({"input": "Fitness, Health, Lifestyle"})

C:\Users\Joshu\AppData\Local\Temp\ipykernel_53560\3236369251.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=final_prompt, llm=creative_llm)


{'input': 'Fitness, Health, Lifestyle', 'text': 'Wellness'}

## Chain of Thought Prompting

Now we want to dive into Chain-Of-Thought prompting, for this there is no direct prompt template however instead what we do is we set the template up to make the AI talk to us about how it will solve the problem, by going through each step rather then just rushing straight to the answer.

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

Here we will settup the step by step rule to enable chain of thought prompting.

In [20]:
# Define the chain-of-thought prompt template
template = """
Question: {question}

First, list systematically and in detail all the problems in this question
that need to be solved before we can arrive at the correct answer.
Then, solve each sub problem using the answers of previous problems
and reach a final solution

Output: Well the right answer is... """

In [21]:
# Define the chain-of-thought prompt template
template2 = """
Question: {question}

Do not use any chain-of-thought processes to answer the question

Output: Well the wrong answer is... """

Then the rest is the same as a simple prompt, where we feed a basic question into the LLM and it will give us a answer.

In [22]:
prompt = PromptTemplate(template=template, input_variables=["question"])
prompt2 = PromptTemplate(template=template2, input_variables=["question"])

In [23]:
# Load the question-answering chain with the chain-of-thought prompt
chain = LLMChain(prompt = prompt, llm = llm)
# Load the question-answering chain with the chain-of-thought prompt
chain2 = LLMChain(prompt = prompt2, llm = llm)

In [24]:
# Ask a question
question = "James has 7 apples, he eats 4 and is given an additional 19 apples, James gives 15 apples to Josh, and Josh gives James 2 apples, how many apples does James have?" 

In [25]:
result = chain.run(question)

print(result)

C:\Users\Joshu\AppData\Local\Temp\ipykernel_53560\2242271788.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(question)


Here are the problems that need to be solved systematically and in detail:

1. James has 7 apples initially.
2. James eats 4 apples.
3. James receives an additional 19 apples.
4. James gives 15 apples to Josh.
5. James gives 2 apples to Josh.

To solve this problem, we will follow these steps:

**Step 1: Calculate the number of apples James has after eating 4**

James starts with 7 apples and eats 4, so he is left with:
7 - 4 = 3

**Step 2: Add the additional 19 apples that James receives**

Now, James has 3 apples and receives an additional 19 apples. So, we add these two numbers together:
3 + 19 = 22

**Step 3: Subtract the number of apples James gives to Josh**

James gives 15 apples to Josh, so he is left with:
22 - 15 = 7

**Step 4: Add the number of apples James gives to Josh**

James gives 2 apples to Josh, so we add these two numbers together:
7 + 2 = 9

Therefore, the correct answer is: James has 9 apples.


In [26]:
result = chain2.run(question)

print(result)

7 - 4 + 19 + 0 (Josh's apples) + 15 + 2 = 23


As you can see with the amount of steps involved without using a step by step guide, the AI can step over important information that would help it achieve the correct results, however it's important to note that without a super long question the AI can solve alot of these issues without using chain prompting, however it's just in these certain instances where chain prompting can be very useful.